<a href="https://colab.research.google.com/github/syed-zeeshan-haider/Rag_project_2.ipynb/blob/main/Rag_project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_community pypdf -q langchain_google_genai  langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 339.7 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 37.3 MB/s eta 0:0

In [2]:
!pip install  --upgrade -q google-auth google-auth-oauthlib google-auth-httplib2  google-cloud


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 kB 4.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.37.0 which is incompatible.


#1 . **FIIRST** YOU NEED TO DOWNLOAD DESRIED PDF FILE AND UPLOAD IT TO COLAB
HERE IS THE FILE LINK : https://drive.google.com/file/d/1D85ntKHxp6zhJerqFbGe4OZh_NkpZRBb/view?usp=sharing


# 2 . Loading Pdf

In [4]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/NIPS-2017-attention-is-all-you-need-Paper.pdf")
data = loader.load()


In [5]:
len(data)

11

# 3 . Spliting text

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("total number of documents : " ,len(docs))

total number of documents :  43


# 4 . Using API key

In [3]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY_3')

##Step 2 Splitting Data

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap = 1000 )
docs = text_splitter.split_documents(data)




# 5 . HERE YOU NEED TO DOWNLOAD ANOTHER FILE AND UPLOAD IT TO COLBA :
HERE IS THE LINK : https://drive.google.com/file/d/1afKObnhCZ9UeXyMuqiV2t4BJ9Nkvgm4D/view?usp=sharing

# 6 . Using json file

In [10]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/astute-expanse-447016-t0-97ea19bbadd9.json"


In [11]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma

# 8 . Creating embedding model

In [12]:
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001" , google_api_key=GOOGLE_API_KEY)

# 9 . Embedding data and Retriver

In [13]:
vectorstore = Chroma.from_documents(documents=docs, embedding=embedding)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})


# 10 . Creating LLM

In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash-exp" , temperature = 0.3 , max_tokens=1000)

# 11 . Defining user_prompt

In [15]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate




system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# 12 . Creating chains

In [16]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# 12 . Asking question which is not in the pdf




In [19]:
response = rag_chain.invoke({"input" : "who is Zeeshan"})
print(response["answer"])

I am sorry, but there is no mention of anyone named Zeeshan in the provided text. Therefore, I cannot answer your question.



# 13 . Now We will Ask question which is  in the pdf

In [20]:
response = rag_chain.invoke({"input" : "what is multi attention"})
print(response["answer"])

Multi-head attention involves projecting queries, keys, and values multiple times with different linear projections. Then, the attention function is performed in parallel on each of these projected versions. This results in multiple dv-dimensional output values which are then concatenated and linearly transformed.

